In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import json 
from PIL import Image
import os
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader


In [ ]:
#cfg
BATCH = 50
EPOCHS = 20

LR = 0.001
IM_SIZE = 256

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

TRAIN_DIR = '../input/cassava-leaf-disease-classification/train_images/'
TEST_DIR = '../input/cassava-leaf-disease-classification/test_images/'

In [ ]:
labels = json.load(open("../input/cassava-leaf-disease-classification/label_num_to_disease_map.json"))
print(labels)

In [ ]:
train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')

In [ ]:
train.head()

In [ ]:
# model = torchvision.models.resnet18()
# model.fc = nn.Linear(2048 , 5, bias=True)

In [ ]:
gpumodel = model.cuda()

In [ ]:
next(gpumodel.parameters()).is_cuda

In [ ]:
X_Train, Y_Train = train['image_id'].values, train['label'].values
X_Test = [name for name in (os.listdir(TEST_DIR))]






In [ ]:
len(Y_Train)


In [ ]:
class GetData(Dataset):
    def __init__(self , Dir , Fnames , Labels , Transform):
        self.dir = Dir
        self.fnames = Fnames
        self.transform = Transform
        self.lbs = Labels
        
    def __len__(self):
        return len(self.fnames)
    
    def __getitem__(self, index):
        x = Image.open(os.path.join(self.dir, self.fnames[index]))
        if "train" in self.dir:            
            return self.transform(x), self.lbs[index]            
        elif "test" in self.dir:            
            return self.transform(x), self.fnames[index]

In [ ]:
Transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((IM_SIZE, IM_SIZE)),
    transforms.RandomRotation(90),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [ ]:
train_data = GetData(TRAIN_DIR, X_Train, Y_Train, Transform)
train_loader = DataLoader(train_data, batch_size = BATCH, shuffle = True, num_workers = 4)

test_data = GetData(TEST_DIR, X_Test, None, Transform)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False, num_workers=4)

In [ ]:
#model


In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model = torchvision.models.resnet152()
model.fc = nn.Linear(2048, 5, bias=True)

In [ ]:
#model
next(model.parameters()).is_cuda


In [ ]:
next(gpumodel.parameters()).is_cuda

In [ ]:
model.layer4[0].conv1

In [ ]:
# torch.manual_seed(42)
# model.classifier = nn.Sequential(nn.Linear(9216, 1024),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.4),
#                                  nn.Linear(1024, 2),
#                                  nn.LogSoftmax(dim=1))
# model

In [ ]:
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f'{item:>8}')
    print(f'________\n{sum(params):>8}')

In [ ]:
count_parameters(model)

In [ ]:


model = model.to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
#train

import time
start_time = time.time()

train_losses = []
test_losses = []
train_correct = []
test_correct = []
EPOCHS = 5
for i in range(EPOCHS):
    trn_corr = 0
    tst_corr = 0
    
    for b,(X_TRAIN, Y_TRAIN) in enumerate(train_loader):
        b+=1
        X_TRAIN = X_TRAIN.to(DEVICE)
        Y_TRAIN = Y_TRAIN.to(DEVICE)
        
        #the model forward pass
        y_pred = model(X_TRAIN)
        loss = criterion(y_pred, Y_TRAIN)
        
        predicted = torch.max(y_pred.data,1)[1]
        batch_corr = (predicted == Y_TRAIN).sum()
        trn_corr += batch_corr
        
        optimizer.zero_grad()  #doubtful
        loss.backward()
        optimizer.step()
        
        if b %100 == 0:
            print(f'epoch: {i:2} batch: {b:4} [{10*b:6}/50000] loss: {loss.item():10.8f}  \
accuracy: {trn_corr.item()*100/(50*b):7.3f}%')
            
            
            
            
    train_losses.append(loss)
    train_correct.append(trn_corr)
            
    
    
    
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed 
    









total_time = time.time() - start_time


In [ ]:
torch.save(model.state_dict(), 'resnetmodel.pt')

In [ ]:
import os
os.getcwd()

In [ ]:
model2 = torchvision.models.resnet152()
model2.fc = nn.Linear(2048, 5, bias=True)
model2.load_state_dict(torch.load('/kaggle/working/resnetmodel.pt'))
#model2.eval()


In [ ]:
model2 = model2.to(DEVICE)

## 

In [ ]:
import time
start_time = time.time()

train_losses = []
test_losses = []
train_correct = []
test_correct = []
EPOCHS = 5
for i in range(EPOCHS):
    trn_corr = 0
    tst_corr = 0
    
    for b,(X_TRAIN, Y_TRAIN) in enumerate(train_loader):
        b+=1
        X_TRAIN = X_TRAIN.to(DEVICE)
        Y_TRAIN = Y_TRAIN.to(DEVICE)
        
        #the model forward pass
        y_pred = model(X_TRAIN)
        loss = criterion(y_pred, Y_TRAIN)
        
        predicted = torch.max(y_pred.data,1)[1]
        batch_corr = (predicted == Y_TRAIN).sum()
        trn_corr += batch_corr
        
        optimizer.zero_grad()  #doubtful
        loss.backward()
        optimizer.step()
        
        if b %100 == 0:
            print(f'epoch: {i:2} batch: {b:4} [{10*b:6}/50000] loss: {loss.item():10.8f}  \
accuracy: {trn_corr.item()*100/(50*b):7.3f}%')
            
            
            
            
    train_losses.append(loss)
    train_correct.append(trn_corr)
            
    
    
    
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed 
    









total_time = time.time() - start_time